In [1]:
import pandas as pd

df = pd.read_csv("datasets/original.csv")
df = df.sort_values(by=["student_id", "period", "discipline"])

print(df.head())

       id  student_id                      birthdate sex                city  \
2138    1       99951  1992-05-21 02:00:00.000 -0300   F  Faxinal do Soturno   
5665    3       99951  1992-05-21 02:00:00.000 -0300   F  Faxinal do Soturno   
9160    2       99951  1992-05-21 02:00:00.000 -0300   F  Faxinal do Soturno   
28710   4       99955  1989-01-06 04:00:00.000 -0200   M  Faxinal do Soturno   
28312   8       99955  1989-01-06 04:00:00.000 -0200   M  Faxinal do Soturno   

                   course  period week_day  \
2138   Ciências Contábeis  2021/1   Quarta   
5665   Ciências Contábeis  2021/2  Segunda   
9160   Ciências Contábeis  2022/1    Terça   
28710  Ciências Contábeis  2025/1  Segunda   
28312  Ciências Contábeis  2025/1   Quarta   

                                 discipline    status   g1   g2  final_grade  \
2138               Matemática Aplicada - 60  Aprovado  7.5  7.0          7.2   
5665       Contabilidade Intermediária - 60  Aprovado  7.2  8.9          8.0   
91

In [2]:
print(f"Quantidade de registros: {len(df)}")

Quantidade de registros: 35995


## Tratamento e Engenharia de Features
- [ ] Criação da coluna binária "canceled" para status "Cancelado" e "Trancado";
- [ ] Criação da coluna binária "skiped_discipline" para status "Reprovado por Frequência";
- [ ] Normalização da coluna "status";
- [ ] Tratamento da coluna "discipline";
- [ ] Exclusão de disciplinas menos cursadas (menos de 10);
- [ ] Normalização da coluna "g1";
- [ ] Normalização da coluna "g2";
- [ ] Normalização da coluna "final_grade";
- [ ] Normalização da coluna "course" (Sistemas de Informação = 1, Direito = 2, ...);
- [ ] Normalização da coluna "week_day" (Segunda = 1, Terça = 2, ...);
- [ ] Normalização da coluna "period" (2025/1 = 1, 2025/2 = 2, ...) obs: cuidar períodos letivos de férias (2025/F1), decair para o período letivo mais próximo?;
- [ ] Normalização da coluna "sex" (M = 1, F = 2);
- [ ] Tratamento da coluna "birthdate", remoção do timestamp, apenas data;
- [ ] Tratamento da coluna "class_skips", dividir valores por 4 (períodos de aula) e normalizar;

OBS: Replicar tratamento e engenharia com as colunas novas e antigas para validação posterior e então dataset final com as originais tratadas (original_and_treated.csv & treated.csv)!

## Criação de Novos Datasets
- [ ] Dataset de alunos de SISTEMAS DE INFORMAÇÃO (registro por aluno) com a listagem total de todas as cadeiras para cada (nome, nota, status, canceled, course, week_day) (information_systems_students.csv)

In [ ]:


# # Tratamento da coluna "status"
# df['status'] = df['status'].astype(str).str.lower().eq('aprovado').astype('int8')

# # Normalização da coluna "final_grade"
# df['final_grade'] = pd.to_numeric(df['final_grade'], errors='coerce')  # valores inválidos viram NaN
# df['final_grade'] = (df['final_grade'] / 10).round(2)
# df.loc[df['final_grade'] == -0.1, 'final_grade'] = 0

# # Remoção do "traço" e do crédito da coluna disciplina no nome da mesma
# df["discipline"] = df["discipline"].str.split(" - ").str[0].str.strip()
# df["discipline"] = df["discipline"].str.split(" -72").str[0].str.strip() # Remoção para caso especial em "Gestão da Qualidade de Software"

# # Anonimização de disciplinas menos cursadas
# discipline_counts = df.groupby("discipline")["student_id"].nunique()
# rare_disciplines = discipline_counts[discipline_counts <= 10].index
# df["discipline"] = df["discipline"].where(~df["discipline"].isin(rare_disciplines), "Outro")

# # Salva dataset atual filtrado apenas para Sistemas de Informação
# df.to_csv("datasets/sistemas_de_informacao.csv", index=False, encoding="utf-8")

# print(f"Quantidade de registros: {len(df)}\n")
# print(df.head(20))